In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import BiCuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'bicudnngrulast')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'bicudnngrulast_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem.pkl')
RESULT_CSV = os.path.join(RESULTS_PATH, 'bigru.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2893 - acc: 0.9127 - val_loss: 0.1963 - val_acc: 0.9452


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.57376, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_0.h5
Epoch 2/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.1956 - acc: 0.9425 - val_loss: 0.1861 - val_acc: 0.9463

Epoch 00002: _f1_monitor improved from 0.57376 to 0.58451, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_0.h5
Epoch 3/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.1808 - acc: 0.9452 - val_loss: 0.1881 - val_acc: 0.9449

Epoch 00003: _f1_monitor improved from 0.58451 to 0.60834, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_0.h5
Epoch 4/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.1659 - acc: 0.9488 - val_loss: 0.1878 - val_acc: 0.9456

Epoch 00004: _f1_monitor im

Epoch 3/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2584 - acc: 0.9157 - val_loss: 0.2704 - val_acc: 0.9127

Epoch 00003: _f1_monitor improved from 0.59099 to 0.59371, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_3.h5
Epoch 4/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2399 - acc: 0.9232 - val_loss: 0.2856 - val_acc: 0.9071

Epoch 00004: _f1_monitor did not improve from 0.59371
Epoch 5/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2176 - acc: 0.9306 - val_loss: 0.2882 - val_acc: 0.9094

Epoch 00005: _f1_monitor did not improve from 0.59371
Epoch 6/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.1932 - acc: 0.9383 - val_loss: 0.3159 - val_acc: 0.9059

Epoch 00006: _f1_monitor did not improve from 0.59371

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



105000/105000 [==============================] - 200s 2ms/step - loss: 0.3534 - acc: 0.8700 - val_loss: 0.4336 - val_acc: 0.8377

Epoch 00004: _f1_monitor did not improve from 0.75523
Epoch 5/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.3204 - acc: 0.8826 - val_loss: 0.4435 - val_acc: 0.8335

Epoch 00005: _f1_monitor did not improve from 0.75523

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2506 - acc: 0.9122 - val_loss: 0.4793 - val_acc: 0.8300

Epoch 00006: _f1_monitor did not improve from 0.75523
Epoch 7/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2302 - acc: 0.9197 - val_loss: 0.5099 - val_acc: 0.8220

Epoch 00007: _f1_monitor did not improve from 0.75523
Epoch 8/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2145 - acc: 0.9262 - val_loss: 0.5145 - val_acc: 0.8229

Epoc


Epoch 00001: _f1_monitor improved from -inf to 0.65370, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_8.h5
Epoch 2/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.3334 - acc: 0.8806 - val_loss: 0.3451 - val_acc: 0.8707

Epoch 00002: _f1_monitor did not improve from 0.65370
Epoch 3/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.3136 - acc: 0.8877 - val_loss: 0.3424 - val_acc: 0.8793

Epoch 00003: _f1_monitor did not improve from 0.65370
Epoch 4/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2911 - acc: 0.8962 - val_loss: 0.3422 - val_acc: 0.8824

Epoch 00004: _f1_monitor improved from 0.65370 to 0.65953, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_8.h5
Epoch 5/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2975 - acc: 0.893

105000/105000 [==============================] - 200s 2ms/step - loss: 0.3591 - acc: 0.8679 - val_loss: 0.4143 - val_acc: 0.8448

Epoch 00004: _f1_monitor did not improve from 0.68224
Epoch 5/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.3284 - acc: 0.8788 - val_loss: 0.4329 - val_acc: 0.8415

Epoch 00005: _f1_monitor did not improve from 0.68224
Epoch 6/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2931 - acc: 0.8934 - val_loss: 0.4483 - val_acc: 0.8383

Epoch 00006: _f1_monitor did not improve from 0.68224

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2204 - acc: 0.9229 - val_loss: 0.5121 - val_acc: 0.8240

Epoch 00007: _f1_monitor did not improve from 0.68224
Epoch 8/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.1979 - acc: 0.9311 - val_loss: 0.5367 - val_acc: 0.8215

Epoc

105000/105000 [==============================] - 200s 2ms/step - loss: 0.2736 - acc: 0.9068 - val_loss: 0.9524 - val_acc: 0.7031

Epoch 00009: _f1_monitor did not improve from 0.64142

Epoch 00009: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 14th grain: f1_score - 0.6414167264346089, acc - 0.7651333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 201s 2ms/step - loss: 0.6628 - acc: 0.7056 - val_loss: 0.5751 - val_acc: 0.7493

Epoch 00001: _f1_monitor improved from -inf to 0.67285, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/bicudnngrulast/bicudnngrulast_15.h5
Epoch 2/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.5482 - acc: 0.7643 - val_loss: 0.5626 - val_acc: 0.7568

Epoch 00002: _f1_monitor improved from 0.67285 to 0.68460, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrn

105000/105000 [==============================] - 200s 2ms/step - loss: 0.4192 - acc: 0.8461 - val_loss: 0.5444 - val_acc: 0.7984

Epoch 00004: _f1_monitor did not improve from 0.55814
Epoch 5/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.3740 - acc: 0.8652 - val_loss: 0.5733 - val_acc: 0.7901

Epoch 00005: _f1_monitor did not improve from 0.55814

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2837 - acc: 0.9034 - val_loss: 0.6386 - val_acc: 0.7755

Epoch 00006: _f1_monitor did not improve from 0.55814
Epoch 7/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2573 - acc: 0.9134 - val_loss: 0.6713 - val_acc: 0.7691

Epoch 00007: _f1_monitor did not improve from 0.55814
Epoch 8/100
105000/105000 [==============================] - 200s 2ms/step - loss: 0.2363 - acc: 0.9214 - val_loss: 0.7040 - val_acc: 0.7740

Epoc

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6334887758671351
The 1th f1: 0.5060042883510675
The 2th f1: 0.6632163688623852
The 3th f1: 0.5937082612287429
The 4th f1: 0.7552329631772543
The 5th f1: 0.6892349797057193
The 6th f1: 0.6934820195111882
The 7th f1: 0.7291529116337684
The 8th f1: 0.6595296686299834
The 9th f1: 0.6193966789469866
The 10th f1: 0.6631097908611521
The 11th f1: 0.6822439030621725
The 12th f1: 0.6902268357797665
The 13th f1: 0.6707066735995137
The 14th f1: 0.6414167264346089
The 15th f1: 0.6845963428241035
The 16th f1: 0.5105833874502437
The 17th f1: 0.7005963558070042
The 18th f1: 0.5581370766661244
The 19th f1: 0.6742776687117109
The average f1 of val is 0.6509170838555315


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)